## Step 1: Import Libraries

In [1]:
from datetime import datetime
import polars as pl
import pandas as pd
import pyarrow as pa
from typing import List
import duckdb
import yfinance as yf

## Step 2: Import Symbols

In [2]:
def load_symbols(file_path: str) -> List[str]:
    """Load symbols from a text file"""
    try:
        with open(file_path, 'r') as f:
            symbols = [line.strip() for line in f if line.strip()]
        print(f"Loaded {len(symbols)} symbols from {file_path}")
        return symbols
    except Exception as e:
        print(f"Error loading symbols: {str(e)}")
        return []

symbols_file = '../tickers.txt'
symbols = load_symbols(symbols_file)

if symbols:
    print("Symbols:", symbols)
else:
    print("No symbols loaded.")

Loaded 61 symbols from ../tickers.txt
Symbols: ['AAPL', 'MSFT', 'GOOGL', 'GOOG', 'META', 'NVDA', 'AVGO', 'ORCL', 'CRM', 'ACN', 'ADBE', 'CSCO', 'INTC', 'NFLX', 'DIS', 'CMCSA', 'VZ', 'T', 'AMZN', 'TSLA', 'HD', 'MCD', 'NKE', 'SBUX', 'TGT', 'LOW', 'WMT', 'PG', 'KO', 'PEP', 'COST', 'BRK-B', 'JPM', 'BAC', 'WFC', 'GS', 'MS', 'BLK', 'UNH', 'JNJ', 'PFE', 'ABBV', 'MRK', 'LLY', 'CAT', 'BA', 'HON', 'UPS', 'RTX', 'GE', 'XOM', 'CVX', 'COP', 'SLB', 'LIN', 'APD', 'ECL', 'PLD', 'AMT', 'CCI', 'OSW']


## Step 3: Extract Data from Yfinance into Pandas

In [3]:
start_date = '2020-01-01'
end_date = datetime.today().strftime('%Y-%m-%d')

# Download prices from yfinance
prices = yf.download(symbols, start=start_date, end=end_date, group_by='ticker')

# Check if the data has a MultiIndex (due to multiple symbols)
if isinstance(prices.columns, pd.MultiIndex):
    # Flatten the MultiIndex columns
    prices.columns = ['_'.join(filter(None, map(str, col))) for col in prices.columns]

# Reset index and melt the DataFrame to include a "symbol" column
prices = prices.copy()  # Avoid fragmentation issues
prices.reset_index(inplace=True)
prices = prices.melt(id_vars=["Date"], var_name="Metric", value_name="Value")
prices[["Symbol", "Metric"]] = prices["Metric"].str.extract(r'([^_]+)_(.+)')
prices = prices.pivot(index=["Date", "Symbol"], columns="Metric", values="Value").reset_index()

display(prices)

[                       0%                       ]

[*                      3%                       ]  2 of 61 completed

[**                     5%                       ]  3 of 61 completed

[***                    7%                       ]  4 of 61 completed

[****                   8%                       ]  5 of 61 completed

...

[**********************95%*********************  ]  58 of 61 completed

[**********************97%********************** ]  59 of 61 completed

[**********************98%********************** ]  60 of 61 completed

[*********************100%***********************]  61 of 61 completed

Metric,Date,Symbol,Close,High,Low,Open,Volume
0,2020-01-02,AAPL,72.796028,72.856621,71.545395,71.799881,135480400.0
1,2020-01-02,ABBV,71.589783,71.605769,70.758365,71.214045,5639200.0
2,2020-01-02,ACN,195.263565,196.908187,194.018490,195.923277,2431100.0
3,2020-01-02,ADBE,334.429993,334.480011,329.170013,330.000000,1990100.0
4,2020-01-02,AMT,200.433929,202.758431,200.004110,201.468987,1426000.0
...,...,...,...,...,...,...,...
77404,2025-01-17,UPS,129.729996,130.580002,129.429993,130.149994,4260300.0
77405,2025-01-17,VZ,38.779999,38.970001,38.349998,38.419998,23988300.0
77406,2025-01-17,WFC,77.080002,77.250000,75.500000,75.980003,20610100.0
77407,2025-01-17,WMT,91.940002,92.260002,91.050003,92.070000,15706100.0


## Step 4: Convert Pandas to Polars

In [4]:
# Convert to Polars DataFrame
df = pl.from_pandas(prices)

# View the Polars DataFrame
print(df)

shape: (77_409, 7)
┌─────────────────────┬────────┬────────────┬────────────┬────────────┬────────────┬────────────┐
│ Date                ┆ Symbol ┆ Close      ┆ High       ┆ Low        ┆ Open       ┆ Volume     │
│ ---                 ┆ ---    ┆ ---        ┆ ---        ┆ ---        ┆ ---        ┆ ---        │
│ datetime[ns]        ┆ str    ┆ f64        ┆ f64        ┆ f64        ┆ f64        ┆ f64        │
╞═════════════════════╪════════╪════════════╪════════════╪════════════╪════════════╪════════════╡
│ 2020-01-02 00:00:00 ┆ AAPL   ┆ 72.796028  ┆ 72.856621  ┆ 71.545395  ┆ 71.799881  ┆ 1.354804e8 │
│ 2020-01-02 00:00:00 ┆ ABBV   ┆ 71.589783  ┆ 71.605769  ┆ 70.758365  ┆ 71.214045  ┆ 5.6392e6   │
│ 2020-01-02 00:00:00 ┆ ACN    ┆ 195.263565 ┆ 196.908187 ┆ 194.01849  ┆ 195.923277 ┆ 2.4311e6   │
│ 2020-01-02 00:00:00 ┆ ADBE   ┆ 334.429993 ┆ 334.480011 ┆ 329.170013 ┆ 330.0      ┆ 1.9901e6   │
│ 2020-01-02 00:00:00 ┆ AMT    ┆ 200.433929 ┆ 202.758431 ┆ 200.00411  ┆ 201.468987 ┆ 1.426e6    │
│

## Step 5: Write Polars to Parquet

In [5]:
output_dir = "../../../data/finance"

# Write DataFrame to Parquet
df.write_parquet(f'{output_dir}/historical_stock_quotes_{start_date}_to_{end_date}.parquet')

## Step 6: Read Parquet (Validate)

In [6]:
pl.scan_parquet(f'{output_dir}/historical_stock_quotes_{start_date}_to_{end_date}.parquet').head().collect()

Date,Symbol,Close,High,Low,Open,Volume
datetime[ns],str,f64,f64,f64,f64,f64
2020-01-02 00:00:00,"""AAPL""",72.796028,72.856621,71.545395,71.799881,1.354804e8
2020-01-02 00:00:00,"""ABBV""",71.589783,71.605769,70.758365,71.214045,5.6392e6
2020-01-02 00:00:00,"""ACN""",195.263565,196.908187,194.01849,195.923277,2.4311e6
2020-01-02 00:00:00,"""ADBE""",334.429993,334.480011,329.170013,330.0,1.9901e6
2020-01-02 00:00:00,"""AMT""",200.433929,202.758431,200.00411,201.468987,1.426e6
